<a href="https://colab.research.google.com/github/harshitrajssss/youtube_chat/blob/main/youtubechat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install langchain openai tiktoken chromadb pypdf sentence_transformers InstructorEmbedding faiss-cpu

In [3]:
pip install openai-whisper

In [1]:
import os
from getpass import getpass
os.environ["OPENAI_API_KEY"]=getpass()

··········


In [2]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

In [3]:
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [8]:
pip install pytube

In [4]:
import whisper
from pytube import YouTube


In [5]:
model=whisper.load_model('base')

In [6]:
Youtube_video_url='https://youtu.be/KyjQJBDD5V4?si=XjodJhbpoyN_F0ue'
Youtube_video=YouTube(Youtube_video_url)

In [7]:
streams=Youtube_video.streams.filter(only_audio=True)
stream=streams.first()
stream.download(filename='cotent.mp4')

'/content/cotent.mp4'

In [8]:
output=model.transcribe('cotent.mp4')['text']
output

" The Inventions The Inventions The Inventions The Inventions Albert Einstein not only is one of history's most important physicists, but his quirky personality and fame made him a cultural icon as well. So much so that Time Magazine even named him person of the century. The inventions that he came up with, the fundamental laws of physics that he figured out, underpin all of our modern day life. He gave rise to the invention of lasers, telecommunication satellites, cell phones, they're everywhere. Albert Einstein was born in Olm, Germany on March 14, 1879, but spent most of his childhood in Munich before his family moved to Milan, Italy. After graduating from the Swiss Polytechnic Institute at age 21, Einstein had a hard time finding employment. Einstein tried to apply for multiple academic positions and actually was turned down because of a recommendation letter that one of his teachers wrote for him. Albert Einstein found full-time work as a patent clerk in Burns, Switzerland in 1902

In [9]:
import os

directory_path = 'book'

if not os.path.exists(directory_path):
    os.makedirs(directory_path)

file_name = 'output.txt'
full_file_path = os.path.join(directory_path, file_name)


with open(full_file_path, 'w') as file:
    file.write(output)

In [10]:
from langchain.document_loaders import TextLoader
loader = TextLoader('/content/book/output.txt')
documents = loader.load()

In [11]:
text_splitter = RecursiveCharacterTextSplitter(
                                               chunk_size=1000,
                                               chunk_overlap=200)

texts = text_splitter.split_documents(documents)

In [12]:
texts[0]

Document(page_content="The Inventions The Inventions The Inventions The Inventions Albert Einstein not only is one of history's most important physicists, but his quirky personality and fame made him a cultural icon as well. So much so that Time Magazine even named him person of the century. The inventions that he came up with, the fundamental laws of physics that he figured out, underpin all of our modern day life. He gave rise to the invention of lasers, telecommunication satellites, cell phones, they're everywhere. Albert Einstein was born in Olm, Germany on March 14, 1879, but spent most of his childhood in Munich before his family moved to Milan, Italy. After graduating from the Swiss Polytechnic Institute at age 21, Einstein had a hard time finding employment. Einstein tried to apply for multiple academic positions and actually was turned down because of a recommendation letter that one of his teachers wrote for him. Albert Einstein found full-time work as a patent clerk in Burns

In [13]:
len(texts)

5

In [14]:
import pickle
import faiss
from langchain.vectorstores import FAISS

In [15]:
def store_embeddings(docs, embeddings, sotre_name, path):

    vectorStore = FAISS.from_documents(docs, embeddings)

    with open(f"{path}/faiss_{sotre_name}.pkl", "wb") as f:
        pickle.dump(vectorStore, f)

In [16]:
def load_embeddings(sotre_name, path):
    with open(f"{path}/faiss_{sotre_name}.pkl", "rb") as f:
        VectorStore = pickle.load(f)
    return VectorStore

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer


In [1]:

Embedding_store_path = "/content/book/Embedding_store"

In [ ]:
db_instructEmbedd = FAISS.from_documents(texts, instructor_embeddings)

In [ ]:
retriever = db_instructEmbedd.as_retriever(search_kwargs={"k": 3})

In [ ]:
qa_chain_instrucEmbed = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0.2, ),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    lines = text.split('\n')


    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]


    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
query = 'what is Qualifications of Chairperson?'
llm_response = qa_chain_instrucEmbed(query)
process_llm_response(llm_response)